<a href="https://colab.research.google.com/github/faezeh-lbf/Probing-Persian-Language-Models/blob/main/persian_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title install and import libs
# !pip install googletrans==3.1.0a0
!pip install jsonlines
from IPython.display import clear_output
import pickle

import jsonlines
import pandas as pd
# from googletrans import Translator
# translator = Translator()
# translated = translator.translate('apple',dest='fa').text
clear_output()
# print('apple: ',translated)


#age comparison

In [ ]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_train.jsonl.gz
!gunzip number_comparison_age_compare_masked_train.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_dev.jsonl.gz
!gunzip number_comparison_age_compare_masked_dev.jsonl.gz


--2021-07-24 23:20:11--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_train.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.88.56
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.88.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99031 (97K) [binary/octet-stream]
Saving to: ‘number_comparison_age_compare_masked_train.jsonl.gz’

number_comparison_a 100%[===================>]  96.71K   507KB/s    in 0.2s    

2021-07-24 23:20:12 (507 KB/s) - ‘number_comparison_age_compare_masked_train.jsonl.gz’ saved [99031/99031]

gzip: number_comparison_age_compare_masked_train.jsonl already exists; do you wish to overwrite (y or n)? n
	not overwritten
--2021-07-24 23:24:50--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_dev.jsonl.gz
Resolving olmpics.s3.

In [ ]:
#@title trans_numbers,large_num2word functions

def num2word(num):
  # if num=='-':
  #   return 'منفی '
  num = int(num)
  result = ''
  word = {1:'یک',2:'دو',3:'سه',4:'چهار',5:'پنج',6:'شش',7:'هفت',8:'هشت',9:'نه',10:'ده',11:'یازده',12:'دوازده',13:'سیزده',14:'چهارده',15:'پانزده',16:'شانزده',17:'هفده',18:'هجده',19:'نوزده',20:'بیست',30:'سی',40:'چهل',50:'پنجاه',60:'شصت',70:'هفتاد',80:'هشتاد',90:'نود',100:'صد',200:'دویست',300:'سیصد',400:'چهارصد',500:'پانصد',600:'ششصد',700:'هفتصد',800:'هشتصد',900:'نهصد'}
  if num==0:
    return 'صفر'
  if num<0:
    result = 'منفی '
    num*=-1
  if num <= 20:
    result+= word[num]
  elif num<100:
    result+=word[10*int(num/10)]
    if num%10>0:
      result+=' و ' + word[num%10]
  elif num<1000:
    result+=word[100*int(num/100)]
    tmp = num%100
    if tmp>0:
      result+=' و '+ num2word(tmp)

  return result

def large_num2word(num):
  if num[0]=='-':
    num=num[1:]
    result='منفی '
  else:
    result=''
  suffix={0:'',1:' هزار',2:' ملیون',3:' میلیارد',4:' تریلیون',5:' تریلیارد'}
  triples = [num[0:len(num)%3]] if len(num)%3!=0 else []
  triples +=[num[i:i+3] for i in range(len(num)%3, len(num), 3)]
  triples.reverse()
  result +=num2word(triples[-1])+suffix[len(triples)-1]
  for i in range(len(triples)-2,-1,-1):
    n=num2word(triples[i])
    if n!='صفر':
      result+=' و '+n+suffix[i]
  return result

def trans_numbers(num):
  num = num.replace('1','۱')
  num = num.replace('2','۲')
  num = num.replace('3','۳')
  num = num.replace('4','۴')
  num = num.replace('5','۵')
  num = num.replace('6','۶')
  num = num.replace('7','۷')
  num = num.replace('8','۸')
  num = num.replace('9','۹')
  num = num.replace('0','۰')
  return num


In [ ]:
#@title translate

def translate_number_comparison(data):
  res={}
  int_check=[]
  per_key={'A':0, 'B':1, 'younger':'کوچکتر','older':'بزرگتر'}
  res['id']=data['id']
  res['label']=per_key[data['answerKey']]
  choices = []
  for choice in data['question']['choices']:
    choices.append({'label':per_key[choice['label']], 'text':per_key[choice['text']]})
  sentence = data['question']['stem'].split(' ')
  stem = 'یک فرد '+ large_num2word(sentence[1]) + ' ساله [MASK] از یک فرد '+large_num2word(sentence[10]) +' ساله است.'
  res['question']={'choices':choices, 'stem':stem}
  return res


with jsonlines.open('number_comparison_age_compare_masked_train.jsonl') as f:
  train=[]
  for line in f.iter():
    train.append(translate_number_comparison(line))

with jsonlines.open('number_comparison_age_compare_masked_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    dev.append(translate_number_comparison(line))


In [ ]:
## @title translate_noLang
def translate_number_comparison(data):
  res={}
  int_check=[]
  per_key={'A':0, 'B':1, 'younger':'محمد','older':'علی'}
  res['id']=data['id']
  res['label']=per_key[data['answerKey']]
  choices = []
  for choice in data['question']['choices']:
    choices.append({'label':per_key[choice['label']], 'text':per_key[choice['text']]})
  sentence = data['question']['stem'].split(' ')
  stem = large_num2word(sentence[1]) + ' [MASK] '+large_num2word(sentence[10])
  res['question']={'choices':choices, 'stem':stem}
  return res


with jsonlines.open('number_comparison_age_compare_masked_train.jsonl') as f:
  train=[]
  for line in f.iter():
    train.append(translate_number_comparison(line))

with jsonlines.open('number_comparison_age_compare_masked_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    dev.append(translate_number_comparison(line))


In [ ]:
#@title save file

with jsonlines.open('mbert_age_comp_train.jsonl', mode='w') as writer:
    writer.write_all(train)
with jsonlines.open('mbert_age_comp_dev.jsonl', mode='w') as writer:
    writer.write_all(dev)

#number comparison

In [ ]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_train.jsonl.gz
!gunzip number_comparison_age_compare_masked_train.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_dev.jsonl.gz
!gunzip number_comparison_age_compare_masked_dev.jsonl.gz


--2021-07-24 23:20:10--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_train.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.88.56
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.88.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99031 (97K) [binary/octet-stream]
Saving to: ‘number_comparison_age_compare_masked_train.jsonl.gz’

number_comparison_a 100%[===================>]  96.71K   506KB/s    in 0.2s    

2021-07-24 23:20:11 (506 KB/s) - ‘number_comparison_age_compare_masked_train.jsonl.gz’ saved [99031/99031]

--2021-07-24 23:20:11--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/number_comparison/number_comparison_age_compare_masked_dev.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.88.56
Connecting to olmpics.s3.us-east-2.amazonaws.com

In [ ]:
def replace_num(sent):
  sent = sent.replace('1','۱')
  sent = sent.replace('2','۲')
  sent = sent.replace('3','۳')
  sent = sent.replace('4','۴')
  sent = sent.replace('5','۵')
  sent = sent.replace('6','۶')
  sent = sent.replace('7','۷')
  sent = sent.replace('8','۸')
  sent = sent.replace('9','۹')
  sent = sent.replace('0','۰')
  return sent


In [ ]:
#@title trans_numbers,large_num2word functions

# def num2word(num):
#   # if num=='-':
#   #   return 'منفی '
#   num = int(num)
#   result = ''
#   word = {1:'یک',2:'دو',3:'سه',4:'چهار',5:'پنج',6:'شش',7:'هفت',8:'هشت',9:'نه',10:'ده',11:'یازده',12:'دوازده',13:'سیزده',14:'چهارده',15:'پانزده',16:'شانزده',17:'هفده',18:'هجده',19:'نوزده',20:'بیست',30:'سی',40:'چهل',50:'پنجاه',60:'شصت',70:'هفتاد',80:'هشتاد',90:'نود',100:'صد',200:'دویست',300:'سیصد',400:'چهارصد',500:'پانصد',600:'ششصد',700:'هفتصد',800:'هشتصد',900:'نهصد'}
#   if num==0:
#     return 'صفر'
#   if num<0:
#     result = 'منفی '
#     num*=-1
#   if num <= 20:
#     result+= word[num]
#   elif num<100:
#     result+=word[10*int(num/10)]
#     if num%10>0:
#       result+=' و ' + word[num%10]
#   elif num<1000:
#     result+=word[100*int(num/100)]
#     tmp = num%100
#     if tmp>0:
#       result+=' و '+ num2word(tmp)

#   return result

# def large_num2word(num):
#   if num[0]=='-':
#     num=num[1:]
#     result='منفی '
#   else:
#     result=''
#   suffix={0:'',1:' هزار',2:' ملیون',3:' میلیارد',4:' تریلیون',5:' تریلیارد'}
#   triples = [num[0:len(num)%3]] if len(num)%3!=0 else []
#   triples +=[num[i:i+3] for i in range(len(num)%3, len(num), 3)]
#   triples.reverse()
#   result +=num2word(triples[-1])+suffix[len(triples)-1]
#   for i in range(len(triples)-2,-1,-1):
#     n=num2word(triples[i])
#     if n!='صفر':
#       result+=' و '+n+suffix[i]
#   return result

# def trans_numbers(num):
#   num = num.replace('1','۱')
#   num = num.replace('2','۲')
#   num = num.replace('3','۳')
#   num = num.replace('4','۴')
#   num = num.replace('5','۵')
#   num = num.replace('6','۶')
#   num = num.replace('7','۷')
#   num = num.replace('8','۸')
#   num = num.replace('9','۹')
#   num = num.replace('0','۰')
#   return num


In [ ]:
#@title translate

def translate_number_comparison(data):
  res={}
  int_check=[]
  per_key={'A':0, 'B':1, 'younger':'کوچکتر','older':'بزرگتر'}
  res['id']=data['id']
  res['label']=per_key[data['answerKey']]
  choices = []
  for choice in data['question']['choices']:
    choices.append({'label':per_key[choice['label']], 'text':per_key[choice['text']]})
  sentence = data['question']['stem'].split(' ')
  stem = 'عدد '+ replace_num(sentence[1]) + ' [MASK] از عدد '+replace_num(sentence[10]) +' است.'
  res['question']={'choices':choices, 'stem':stem}
  return res


with jsonlines.open('number_comparison_age_compare_masked_train.jsonl') as f:
  train=[]
  for line in f.iter():
    train.append(translate_number_comparison(line))

with jsonlines.open('number_comparison_age_compare_masked_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    dev.append(translate_number_comparison(line))


In [ ]:
#@title translate_noLang

def translate_number_comparison(data):
  res={}
  int_check=[]
  per_key={'A':0, 'B':1, 'younger':'کاوه','older':'امیر'}
  res['id']=data['id']
  res['label']=per_key[data['answerKey']]
  choices = []
  for choice in data['question']['choices']:
    choices.append({'label':per_key[choice['label']], 'text':per_key[choice['text']]})
  sentence = data['question']['stem'].split(' ')
  stem = replace_num(sentence[1]) + ' [MASK] ' + replace_num(sentence[10]) 
  res['question']={'choices':choices, 'stem':stem}
  return res


with jsonlines.open('number_comparison_age_compare_masked_train.jsonl') as f:
  train=[]
  for line in f.iter():
    train.append(translate_number_comparison(line))

with jsonlines.open('number_comparison_age_compare_masked_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    dev.append(translate_number_comparison(line))


In [ ]:
#@title save file

with jsonlines.open('num_comp_train_nolang.jsonl', mode='w') as writer:
    writer.write_all(train)
with jsonlines.open('num_comp_dev_nolang.jsonl', mode='w') as writer:
    writer.write_all(dev)

#Object comparison

In [ ]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/size_comparison/size_comparison_train.jsonl.gz
!gunzip size_comparison_train.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/size_comparison/size_comparison_dev.jsonl.gz
!gunzip size_comparison_dev.jsonl.gz

--2021-07-24 23:32:04--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/size_comparison/size_comparison_train.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.102.226
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.102.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134685 (132K) [binary/octet-stream]
Saving to: ‘size_comparison_train.jsonl.gz’

size_comparison_tra 100%[===================>] 131.53K   687KB/s    in 0.2s    

2021-07-24 23:32:04 (687 KB/s) - ‘size_comparison_train.jsonl.gz’ saved [134685/134685]

--2021-07-24 23:32:05--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/size_comparison/size_comparison_dev.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.102.226
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.102.226|:443... connected.
HTTP r

In [ ]:
# @title dictionary! 
#niddle=سوزن؟! pan=چه حیوونی؟!
en_fa_dict  = {'pencil':'مداد', 'bike':'دوچرخه', 'mars':'مریخ', 'spoon':'قاشق', 'drinking glass':'لیوان', 'boat':'قایق', 'niddle':'سوزن', 'bench':'نیمکت', 'river':'رودخانه', 'desk':'میز مطالعه', 'mountain':'کوه', 'fork':'چنگال', 'pen':'خودکار', 'battery':'باتری', 'train':'قطار', 'car':'ماشین', 'planet':'سیاره', 'backpack':'کوله‌پشتی', 
             'table':'میز', 'shoe':'لنگه کفش', 'airplane':'هواپیما', 'bed':'تخت خواب', 'seed':'دانه', 'street':'خیابان', 'sun':'خورشید', 'hat':'کلاه', 'earth':'زمین', 'moon':'ماه', 'feather':'پر', 'city':'شهر', 'house plant':'گیاه خانگی', 'nail':'میخ', 'laptop':'لپ‌تاپ', 'plate':'بشقاب', 'house':'خانه',
             
             'salmon': 'ماهی سالمون', 'pony': 'پونی', 'bear': 'خرس', 'alpaca': 'آلپاکا', 'wolf': 'گرگ', 'flea': 'کک', 'dog': 'سگ', 'reindeer': 'گوزن شمالی', 'crocodiles': 'تمساح', 'cat': 'گربه', 'snail': 'حلزون', 'cockapoo': 'کاکاپو', 'elephant': 'فیل', 'boxer': 'سگ باکسر', 'cougar': 'گربه وحشی پشمالو', 'turtle': 'لاک پشت',
              'bulldog': 'بولداگ', 'parrot': 'طوطی', 'cattle': 'گاو', 'hamster': 'همستر', 'great white shark': 'کوسه سفید بزرگ', 'bald eagle': 'عقاب سرسفید', 'camel': 'شتر', 'pointer': 'سگ پوینتر', 'chinook': 'چینیوک', 'goat': 'بز', 'mouse': 'موش', 'brown bear': 'خرس قهوه ای', 'bee': 'زنبور عسل', 'rabbit': 'خرگوش', 'tuna': 'ماهی تن', 
              'coyote': 'کایوت', 'pomeranian': 'سگ پامرانی', 'cod': 'ماهی کاد', 'pelican': 'پلیکان', 'lobster': 'خرچنگ دریایی', 'northern pike': 'پایک شمالی', 'raccoons': 'راکون', 'cheetah': 'یوزپلنگ', 'wildebeest': 'کل یالدار', 'goldendoodle': 'گلدن دودل', 'penguin': 'پنگوئن', 'chipmunk': 'سنجاب راه‌راه', 'horse': 'اسب', 'deer': 'گوزن',
              'toy poodle': 'پودل اسباب بازی', 'tiger': 'ببر', 'dinosaur': 'دایناسور', 'wild boar': 'گراز وحشی', 'sheltie': 'شتلند شیپداگ', 'manatee': 'گاو نر', 'dachshund': 'داشهوند', 'pug': 'پاگ', 'alligators': 'تمساح', 'chamois': 'شوکا', 'guinea pig': 'خوکچه هندی', 'largemouth bass': 'باس دهان‌گشاد', 'pit bull': 'پیت بول', 'billy': 'بز بیلی', 
              'chihuahua': 'چیهوا', 'orangutan': 'اورانگوتان', 'labradoodle': 'لابرادودل', 'baboons': 'باباون', 'poodle': 'پودل', 'raccoon': 'راکون', 'human': 'انسان', 'sturgeon': 'ماهیان خاویاری', 'koala': 'کوالا', 'swallow': 'چلچله', 'blue whale': 'وال آبی', 'mule': 'قاطر', 'anaconda': 'آناکوندا', 'owl': 'جغد', 'sheep': 'گوسفند', 'capybara': 'برگچه‌خوار',
              'common carp': 'کپور معمولی', 'rat terrier': 'تریر موش', 'liger': 'شیرپلنگ', 'golden retriever': 'سگ شکاری طلایی رنگ دورگه', 'moose': 'گوزن شمالی', 'donkey': 'خر', 'sperm whale': 'نهنگ عنبر', 'walrus': 'گراز دریایی', 'tortoise': 'لاک پشت', 'ant': 'مورچه', 'octopus': 'اختاپوس', 'grizzly bear': 'خرس گریزلی', 'collie': 'کولای', 
              'giant squid': 'ماهی مرکب غول پیکر', 'ferret': 'موش خرما', 'lion': 'شیر نر', 'bull shark': 'کوسه گاو نر', 'rat': 'موش', 'shih tzu': 'شی تزو', 'koi': 'کپور گلگون', 'chimpanzee': 'شامپانزه', 'whale shark': 'کوسه نهنگ', 'bat': 'خفاش', 'giraffe': 'زرافه', 'gorilla': 'گوریل', 'polar bear': 'خرس قطبی', 'llama': 'لاما', 'beagle': 'بیگل', 
              'greyhound': 'سگ تازی', 'shark': 'کوسه', 'hummingbird': 'زرین پر', 'jaguar': 'جگوار', 'bobcat': 'گربه وحشي', 'mosquito': 'پشه', 'wombat': 'وومبت', 'mirror carp': 'کپور آینه ای', 'border collie': 'بوردر کالی', 'rottweiler': 'روتوایلر', 'puppy': 'توله سگ', 'hippopotamus': 'اسب ابی', 'leopard': 'پلنگ', 'goldfish': 'ماهی قرمز',
              'killer whale': 'نهنگ قاتل', 'sea lion': 'شیر دریایی', 'humpback whale': 'نهنگ کوهان دار', 'german shepherd': 'ژرمن شپرد', 'chinchilla': 'چینچیلا', 'spider': 'عنکبوت', 'butterfly': 'پروانه', 'elk': 'گوزن', 'pan': 'شامپانزه(پن)'}

In [ ]:
#@title translate

def translate_object_comparison(data):
  res={}
  int_check=[]
  per_key={'A':0, 'B':1, 'smaller':'کوچکتر','larger':'بزرگتر'}
  res['id']=data['id']
  res['label']=per_key[data['answerKey']]
  choices = []
  if len(data['question']['choices'])!=2 or (data['answerKey']!='A' and data['answerKey']!='B') :
    print(data)
    raise 1
  for choice in data['question']['choices']:
    if choice['label']!='A' and choice['label']!='B':
      print(data)
      raise 1
    choices.append({'label':per_key[choice['label']], 'text':per_key[choice['text']]})
  sentence = data['question']['stem'].split('[MASK]')
  word1 = sentence[0].replace('The size of a ','').replace(' is usually much ','')
  word2 = sentence[1].replace(' than the size of a ','').replace(' .','')
  stem = 'ابعاد یک '+ en_fa_dict[word1] + ' معمولا بسیار [MASK] از ابعاد یک ' + en_fa_dict[word2] +' است.'
  res['question']={'choices':choices, 'stem':stem}
  return res

k = 0
with jsonlines.open('size_comparison_train.jsonl') as f:
  train=[]
  for line in f.iter():
    k+=1
    train.append(translate_object_comparison(line))
print(k)
with jsonlines.open('size_comparison_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    dev.append(translate_object_comparison(line))


5000


In [ ]:
#@title translate_nolang

def translate_object_comparison(data):
  res={}
  int_check=[]
  per_key={'A':0, 'B':1, 'smaller':'آیدا','larger':'مهسا'}
  res['id']=data['id']
  res['label']=per_key[data['answerKey']]
  choices = []
  if len(data['question']['choices'])!=2 or (data['answerKey']!='A' and data['answerKey']!='B') :
    print(data)
    raise 1
  for choice in data['question']['choices']:
    if choice['label']!='A' and choice['label']!='B':
      print(data)
      raise 1
    choices.append({'label':per_key[choice['label']], 'text':per_key[choice['text']]})
  sentence = data['question']['stem'].split('[MASK]')
  word1 = sentence[0].replace('The size of a ','').replace(' is usually much ','')
  word2 = sentence[1].replace(' than the size of a ','').replace(' .','')
  stem =en_fa_dict[word1] + ' [MASK] ' + en_fa_dict[word2]
  res['question']={'choices':choices, 'stem':stem}
  return res


with jsonlines.open('size_comparison_train.jsonl') as f:
  train=[]
  for line in f.iter():
    train.append(translate_object_comparison(line))

with jsonlines.open('size_comparison_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    dev.append(translate_object_comparison(line))


In [ ]:
#@title save file

with jsonlines.open('mbert_size_comp_train_nolang.jsonl', mode='w') as writer:
    writer.write_all(train)
with jsonlines.open('mbert_size_comp_dev_nolang.jsonl', mode='w') as writer:
    writer.write_all(dev)

#antonym_synonym

In [ ]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/antonym_synonym_negation/antonym_synonym_negation_train.jsonl.gz
!gunzip antonym_synonym_negation_train.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/antonym_synonym_negation/antonym_synonym_negation_dev.jsonl.gz
!gunzip antonym_synonym_negation_dev.jsonl.gz
clear_output()

In [ ]:
#@title dic for test
s=[]
c=0
with jsonlines.open('antonym_synonym_negation_dev.jsonl') as f:
  for line in f.iter():
    line = line["question"]["stem"]
    # c+=1
    # if c==5:
      # raise 2
    if line[0]=='H':
      line = line.replace("He was [MASK] ready to ","")
      line = line.replace("He was really ready to ","")
      line = line.replace(".","")
      if line.find("[MASK]")!=-1:
        print(line)
        raise 1
      words = line.split(",")
    elif line[0]=='I':
      line = line.replace("It was [MASK] ","")
      line = line.replace("it was really ","")
      line = line.replace(".","")
      if line.find("[MASK]")!=-1:
        raise 1
      words = line.split(",")
    else:
      print(line)
      raise 2

    # words = line[2:].split(" and a ")
    for word in words:
      s.append(word)
# test_dict={w:translator.translate(w,dest='fa').text for w in s}
# translated = translator.translate('apple',dest='fa')
print(len(s))

1000


In [ ]:

# with jsonlines.open('compositional_comparison_train.jsonl') as f:
#   train=[]
#   for line in f.iter():
#     train.append(translate_object_comparison(line))
def translate_ant_syn(data,dic):
  tr={'A':0,'B':1,'really':'محمد','not':'علی'}
  res={}
  # res['id']=data['id']
  res['label']=tr[data['answerKey']]
  data=data['question']
  choices=[]
  for c in data['choices']:
    choices.append({'label':tr[c['label']],'text':tr[c['text']]})
  # res['question']['choices']=choices
  data=data['stem']

  if data[0]=='H':
    data = data.replace("He was [MASK] ready to ","")
    data = data.replace("He was really ready to ","")
    data = data.replace(".","")
    if data.find("[MASK]")!=-1:
      print(data)
      raise 'sentence format is not considered in translation.'
    words = data.split(",")
    # stem='او آماده‌ی '+dic[words[0].strip()]+' بود. او واقعا آماده‌ی '+dic[words[1].strip()]+' [MASK].'
    stem= dic[words[0].strip()]+ ' '+dic[words[1].strip()]+' [MASK]'
  elif data[0]=='I':
    data = data.replace("It was [MASK] ","")
    data = data.replace("it was really ","")
    data = data.replace(".","")
    if data.find("[MASK]")!=-1:
      raise 'sentence format is not considered in translation.'
    words = data.split(",")
    stem = 'آن '+dic[words[0].strip()]+' بود. آن واقعا '+dic[words[1].strip()]+' [MASK].'
  res['question']={'choices':choices,'stem':stem}
  return res



dic={}
s=0
with open('ant-syn_train.txt','r',encoding='utf-16') as f:
    for line in f:
      s+=1
      line=line.split(':')
      dic[line[0].strip()]=line[1].strip()
del_train=0
with jsonlines.open('antonym_synonym_negation_train.jsonl') as f:
  train=[]
  for line in f.iter():
    try:
      train.append(translate_ant_syn(line,dic))
    except KeyError:
      del_train+=1

print('deleted/remaining train data: ',del_train,len(train))

dic={}
s=0
with open('ant-syn_dev.txt','r',encoding='utf-16') as f:
    for line in f:
      s+=1
      line=line.split(':')
      dic[line[0].strip()]=line[1].strip()
del_dev=0
with jsonlines.open('antonym_synonym_negation_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    try:
      dev.append(translate_ant_syn(line,dic))
    except KeyError:
      del_dev+=1

print('deleted/remaining dev data: ',del_dev,len(dev))


FileNotFoundError: ignored

In [ ]:

dic={}
s=0
with open('ant-syn_.txt','r',encoding='utf-16') as f:
    for line in f:
      s+=1
      line=line.split(':')
      dic[line[0].strip()]=line[1].strip()
print(s)

5538


In [ ]:
import random
randomlist = random.sample(range(0, len(dev)), 20)
for i in randomlist:
  print(dev[i])

{'label': 0, 'question': {'choices': [{'label': 0, 'text': 'نبود'}, {'label': 1, 'text': 'بود'}], 'stem': 'آن یک بی توجهی بود. آن واقعا یک ملاحظه [MASK].'}}
{'label': 1, 'question': {'choices': [{'label': 0, 'text': 'نبود'}, {'label': 1, 'text': 'بود'}], 'stem': 'آن یک روح بود. آن واقعا یک روان [MASK].'}}
{'label': 1, 'question': {'choices': [{'label': 0, 'text': 'نبود'}, {'label': 1, 'text': 'بود'}], 'stem': 'آن یک جانشین بود. آن واقعا یک جایگزین [MASK].'}}
{'label': 1, 'question': {'choices': [{'label': 0, 'text': 'بود'}, {'label': 1, 'text': 'نبود'}], 'stem': 'آن بیرونی بود. آن واقعا درونی [MASK].'}}
{'label': 1, 'question': {'choices': [{'label': 0, 'text': 'بود'}, {'label': 1, 'text': 'نبود'}], 'stem': 'آن غیرمسلح بود. آن واقعا مسلح [MASK].'}}
{'label': 0, 'question': {'choices': [{'label': 0, 'text': 'نبود'}, {'label': 1, 'text': 'بود'}], 'stem': 'آن چربی بود. آن واقعا لاغر [MASK].'}}
{'label': 1, 'question': {'choices': [{'label': 0, 'text': 'نبود'}, {'label': 1, 'text': 'بود'}]

In [ ]:
i=123
dev[i],train[i]

In [ ]:
#@title save file
with jsonlines.open('ant-syn_train.jsonl', mode='w') as writer:
    writer.write_all(train)
with jsonlines.open('ant-syn_dev.jsonl', mode='w') as writer:
    writer.write_all(dev)

##[trash]

In [ ]:
ind=0
translated=[]
for i in range(18):
  # for j in range(505):
  translated.append(translator.translate(s[ind:ind+505],dest='fa').text)
  ind+=505
translated.append(translator.translate(s[9090:],dest='fa').text)

AttributeError: ignored

In [ ]:
ind=0
for i in range(2):
  with open('output'+str(i)+'.txt','w') as f:
    for j in range(505):
      f.write(s[ind].strip()+'\n')
      ind+=1
      if ind==1000:
        break

In [ ]:
name='ant-syn_train'
with open(name+'.pkl', 'rb') as f:
  diction =  pickle.load(f)

In [ ]:
fin = {w.strip():diction[w.strip()].strip() for w in s}

In [ ]:
len(fa),len(en)

(9558, 9558)

In [ ]:
len(fa2)

6714

In [ ]:
en2=[]
fa2=[]
for i in range(500):
  en2.append(en[i])
  fa2.append(fa[i])
enset=set(en2)
# faset=set(fa2)
for i in range(500,9558,2):
  if (en[i] not in enset) or (en[i+1] not in enset):
    en2.append(en[i])
    en2.append(en[i+1])
    fa2.append(fa[i])
    fa2.append(fa[i+1])
    enset.add(en[i])
    enset.add(en[i+1])
    

In [ ]:
en=[]
fa=[]
name='ant-syn_dev'
with open(name+'.txt','r') as f:
  for line in f:
    line=line.split(':')
    en.append(line[0].strip())
    fa.append(line[1].strip())

In [ ]:
len(fa)

1000

In [ ]:
s = list(s)
# tr = translator.translate(list(s), dest='fa')
import os
m=[]
for i in range(len(s)):
  if s[i].find()!=-1:
    print(i)

In [ ]:
len(s),len(l)

In [ ]:
ind=0
for i in range(7):
  with open('output'+str(i)+'.txt','w') as f:
    for j in range(500):
      f.write(s[ind].strip()+'\n')
      ind+=1
      if ind==3267:
        break


In [ ]:
name='ant-syn_dev_t'
with open(name+'.txt','w') as f:
  for i in range(len(en2)):
    f.write(en2[i].strip()+' : '+fa2[i].strip()+'\n')

In [ ]:
l=[]
with open('/content/New Text Document (2).txt','r',encoding='utf-16') as f:
    for line in f:
      l.append(line)
len(l)
# l[-2]

1000

In [ ]:
for i in range(len(l)):
  l[i]=l[i].replace("\n","")
len(l)

3267

In [ ]:
import pickle
names = ['ant-syn_dev','ant-syn_train','tax_train','tax_dev']
# diction = {s[i]:l[i] for i in range(len(s))}
for name in names:
  with open('/content/' + name + '.pkl', 'rb') as f:
    diction =  pickle.load(f)
  with open(name+'.txt','w') as f:
    for k in diction:
      f.write(k.strip()+' : '+diction[k].strip()+'\n')



  # with open('tax_dev.pkl','wb') as f:
  #   pickle.dump(diction,f)
  # tax_dev

In [ ]:
# black list

# chang_list
{
    ' 11 ': '۱۱',
 ' 19 ': '۱۹',
 ' 200 ': '۲۰۰',
}

In [ ]:
#@title sizes
print("SIZE OF DATASETS:")
c=0
with jsonlines.open('antonym_synonym_negation_dev.jsonl') as f:
  for line in f.iter():
    c+=1
print('    antonym_synonym_negation_dev.jsonl: ',c)
c=0
with jsonlines.open('antonym_synonym_negation_train.jsonl') as f:
  for line in f.iter():
    c+=1
print('    antonym_synonym_negation_train.jsonl: ',c)
c=0
with jsonlines.open('hypernym_conjunction_train.jsonl') as f:
  for line in f.iter():
    c+=1
print('    hypernym_conjunction_train.jsonl: ',c)
c=0
with jsonlines.open('hypernym_conjunction_dev.jsonl') as f:
  for line in f.iter():
    c+=1
print('    hypernym_conjunction_dev.jsonl: ',c)

SIZE OF DATASETS:
    antonym_synonym_negation_dev.jsonl:  500
    antonym_synonym_negation_train.jsonl:  4779
    hypernym_conjunction_train.jsonl:  5310
    hypernym_conjunction_dev.jsonl:  599


#taxonomy

In [ ]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/hypernym_conjunction/hypernym_conjunction_train.jsonl.gz
!gunzip hypernym_conjunction_train.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/hypernym_conjunction/hypernym_conjunction_dev.jsonl.gz
!gunzip hypernym_conjunction_dev.jsonl.gz
clear_output()

In [ ]:
#@title dic for test
s=set()

with jsonlines.open('hypernym_conjunction_dev.jsonl') as f:
  for line in f.iter():
    for c in line['question']['choices']:
      s.add(c["text"])
    line = line["question"]["stem"]
    line = line.replace(" are both a type of [MASK] ","")
    if line.find("[MASK]")!=-1:
      print(line)
      raise 1
    words = line[2:].split(" and a ")
    for word in words:
      s.add(word)
# test_dict={w:translator.translate(w,dest='fa').text for w in s}
# translated = translator.translate('apple',dest='fa')
print(len(s))

497


In [ ]:

# with jsonlines.open('compositional_comparison_train.jsonl') as f:
#   train=[]
#   for line in f.iter():
#     train.append(translate_object_comparison(line))
def translate_exist(data,dic):
  res={}
  res['id']=data['id']
  res['label']=data['answerKey']
  data=data['question']
  choices=[]
  for c in data['choices']:
    choices.append({'label':c['label'],'text':dic[c['text']]})
  res['choices']=choices
  data=data['stem']
  data=data.replace(" are both a type of [MASK] ","")
  types=data[2:].split(" and a ")
  stem = dic[types[0].strip()]+" و "+dic[types[1].strip()]+" هر دو از انواع [MASK] هستند."
  res['stem']=stem
  return res



dic={}
s=0
with open('tax_dev.txt','r',encoding='utf-16') as f:
    for line in f:
      s+=1
      line=line.split(':')
      dic[line[0].strip()]=line[1].strip()

with jsonlines.open('hypernym_conjunction_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    try:
      dev.append(translate_exist(line,dic))
    except KeyError:
      print(line)

In [ ]:
dev

[{'choices': [{'label': 'A', 'text': 'پستاندار'},
   {'label': 'B', 'text': 'پستاندار'},
   {'label': 'C', 'text': 'گوزن'}],
  'id': 'a9286e16c2a0e2dc8f9e9354bab2510a',
  'label': 'A',
  'stem': 'گوزن سفید و پاتو هر دو از انواع [MASK] هستند.'},
 {'choices': [{'label': 'A', 'text': 'ماده'},
   {'label': 'B', 'text': 'پستاندار'},
   {'label': 'C', 'text': 'گوزن'}],
  'id': 'd83dee7dfcbcff7b1c0024402c37ae2f',
  'label': 'B',
  'stem': 'گوزن ماده و خرگوش هر دو از انواع [MASK] هستند.'},
 {'choices': [{'label': 'A', 'text': 'پستاندار'},
   {'label': 'B', 'text': 'سگ'},
   {'label': 'C', 'text': 'پستاندار'}],
  'id': 'dc1878a0a003311c9f0c5fa9f3a56443',
  'label': 'A',
  'stem': 'پینچر و کاپوچینو هر دو از انواع [MASK] هستند.'},
 {'choices': [{'label': 'A', 'text': 'ماده'},
   {'label': 'B', 'text': 'سگ'},
   {'label': 'C', 'text': 'پستاندار'}],
  'id': 'e53116f523435609e74ff28cba9da0c3',
  'label': 'C',
  'stem': 'خرگوش و سگ شکاری پا کوتاه هر دو از انواع [MASK] هستند.'},
 {'choices': [{'label'

In [ ]:
ind=0
en=list(s)
for i in range(1):
  with open('output'+str(i)+'.txt','w') as f:
    for j in range(500):
      f.write(en[ind].strip()+'\n')
      ind+=1
      if ind==497:
        break

In [ ]:
fa=[]
with open('/content/New Text Document (2).txt','r',encoding='utf-16') as f:
    for line in f:
      fa.append(line)
len(fa)

497

In [ ]:
name='tax_dev'
with open(name+'.txt','w') as f:
  for i in range(len(s)):
    f.write(en[i].strip()+' : '+fa[i].strip()+'\n')

In [ ]:
# @title print(test_dict)

# {'varietal': 'گونه ای', 'papillon': 'پروانه', 'howler monkey': 'میمون جیغکش', 'soup': 'سوپ', 'roan': 'ناله کردن', 'sambar': 'به سامبا', 'teriyaki': 'تریاکی',
#  'quassia': 'کوازیا', 'praya': 'نماز خواندن', 'fennel': 'رازیانه', 'frank': 'صریح', 'oystercatcher': 'صدف گیر', 'plonk': 'پیرمرد', 'scarab': 'اسکارب', 
#  'goldeneye': 'چشم طلایی', 'kingwood': 'کینگ وود', 'haddock': 'اسکله', 'gallinule': 'گالینول', 'bonito': 'زیبا', 'ocelot': 'ocelot', 'potto': 'گلدان', 
#  'bisque': 'سوپ غلیظ خامه ای', 'cicada': 'سیکادا', 'kava': 'قهوه', 'scarlet tanager': 'دباغ قرمز', 'peacock': 'طاووس', 'tench': 'تنچ', 'agar': 'به طوری که',
#  'hyacinth': 'سنبل', 'collins': 'کولین', 'salmonid': 'ماهی قزل آلا', 'pompon': 'pom POM', 'bouillon': 'ابگوشت', 'applejack': 'جیب سیب', 
#  'pudding': 'پودینگ', 'merganser': 'تاجر', 'kauri': 'کائوری', 'cholesterol': 'کلسترول', 'gudgeon': 'گوزن', 'souffle': 'نفس کشیدن', 'bat': 'یکی', 
#  'sassafras': 'ساسافراس', 'alcohol': 'الکل', 'aperitif': 'اپریتیف', 'tetra': 'تتر', 'bullhead': 'گاوباز', 'tupelo': 'تاپلو', 'scaup': 'اسکاپ کردن',
#  'hooch': 'هوچ', 'scammony': 'کلاهبرداری', 'cypress': 'سرو', 'buckskin': 'پوست مرغابی', 'silver salmon': 'ماهی آزاد', 'cobia': 'کوبیا',
#  'barracouta': 'باراکوتا', 'kelpie': 'کلپی', 'killifish': 'کشتن', 'brazilwood': 'چوب برزیل', 'walleye': 'آهسته', 'walrus': 'گراز دریایی',
#  'marmoset': 'مارموست', 'lasagna': 'لازانیا', 'beluga': 'بلوگا', 'chimpanzee': 'شامپانزه', 'dog': 'سگ', 'cedar waxwing': 'واکس زدن سرو',
#  'mallard': 'اردک کوهی', 'vervet': 'مخمل', 'drill': 'مته', 'earthworm': 'کرم خاکی', 'redpoll': 'گلوله قرمز', 'poodle': 'پودل', 'anthropoid': 'آنتروپوئید',
#  'cichlid': 'سیکلید', 'feist': 'فیست', 'beech': 'راش', 'collie': 'کولای', 'jungle fowl': 'مرغ جنگل', 'escolar': 'مدرسه', 'bock': 'جفتک انداختن',
#  'grouse': 'غرغره', 'turbot': 'توربوت', 'ermine': 'حیوانات کوهی', 'kudu': 'باید', 'springer spaniel': 'spaniger spaniel', 'marabou': 'مارابو', 
#  'thresher': 'خرمن کوب', 'julienne': 'ژولین', 'comma': 'بند', 'liza': 'لیزا', 'burbot': 'بوربوت', 'blastula': 'بلاستول', 'parsley': 'جعفری',
#  'celeriac': 'گلر', 'barbu': 'ریش', 'chitterlings': 'بچه شیرخواران', 'red deer': 'گوزن قرمز', 'papyrus': 'پاپیروس', 'muskellunge': 'ماسکلانژ', 
#  'sea bream': 'ساقه دریایی', 'pricket': 'قارچ', 'leg of lamb': 'پا از گوشت بره', 'halal': 'حلال', 'sterol': 'استرول', 'osprey': 'اسپری', 
#  'sculpin': 'مجسمه سازی', 'cinnabar': 'سنجاب', 'albatross': 'آلباتروس', 'food': 'غذا', 'bongo': 'بونگو', 'geneva': 'ژنو', 'mako': 'ماکو', 
#  'balata': 'آستر', 'malamute': 'نادیده گرفتن', 'grog': 'گورگ', 'benedictine': 'بندیکتین', 'corn earworm': 'کرم گوش ذرت', 'dish': 'ظرف', 
#  'bee': 'زنبور عسل', 'manta': 'مانتو', 'elk': 'هر یک', 'salt': 'نمک', 'curacao': 'کوراکائو', 'pinscher': 'پینچر', 'poon': 'پون', 'satinwood': 'چوب ساتن',
#  'bufo': 'بوفو', 'kino': 'چوبی', 'barracuda': 'باراکودا', 'ern': 'ارن', 'windowpane': 'شیشه پنجره', 'alder': 'سن', 'eggnog': 'تخم مرغ', 
#  'vinegar': 'سرکه', 'nougat': 'نوقا', 'sumac': 'سماق', 'zabaglione': 'zabaglione', 'prosciutto': 'ژامبون', 'quail': 'بلدرچین', 'poteen': 'پوتین', 
#  'finch': 'فنچ', 'pooch': 'پوچ', 'cocker spaniel': 'کوکر اسپانیل', 'underwing': 'تحت', 'confit': 'شیرین', 'consomme': 'مصرف شود', 'chowder': 'چودر',
#  'hog': 'گراز', 'organism': 'ارگانیسم', 'partridge': 'کبک', 'pitta': 'پیتا', 'metheglin': 'متگلین', 'whitebait': 'سفید سفید', 'champagne': 'شامپاین',
#  'chorizo': 'کوریزو', 'borsch': 'بورچ', 'siskin': 'سیسکین', 'wagtail': 'دم جنبانک', 'fryer': 'سرخ کن', 'turmeric': 'زردچوبه', 'diol': 'دیول',
#  'guenon': 'گنون', 'souse': 'مکیدن', 'hot toddy': 'کودک گرمی', 'gorgonzola': 'گورگونزولا', 'shoveler': 'بیلچه زن', 'pika': 'طولانی', 'primate': 'پستاندار',
#  'animal': 'حیوان', 'claret': 'کلار', 'kea': 'کیا', 'bluegill': 'bluegill', 'weevil': 'شپشک', 'menhaden': 'منهدن', 'whooping crane': 'جرثقیل بزرگ',
#  'brocket': 'توپی', 'pepperoni': 'پپرونی', 'banquet': 'ضیافت', 'julep': 'جولپ', 'parrot': 'طوطی', 'cisco': 'سیسکو', 'spitz': 'اسپیتز', 
#  'setter': 'تنظیم کننده', 'beer': 'آبجو', 'barbet': 'باربت', 'bourbon': 'مشروب بربون', 'dachshund': 'داشوند', 'bufflehead': 'سر گاومیش', 
#  'chaetodon': 'چتودون', 'remora': 'بازسازی', 'porgy': 'پورگی', 'moorcock': 'مورکاک', 'peafowl': 'مرغ دریایی', 'tarsier': 'ریزتر', 'wine': 'شراب', 
#  'ricotta': 'پنیر ریکوتا', 'rotgut': 'گندم', 'woad': 'وای', 'lizard': 'مارمولک', 'hoopoe': 'هدهد', 'terrier': 'تریر', 'teak': 'ساج', 
#  'moussaka': 'موساکا', 'redstart': 'شروع مجدد', 'wheatear': 'گندم', 'mascarpone': 'پنیر ماسکارپونه', 'bloodhound': 'سگ خون', 'coho': 'از چی', 
#  'aquavit': 'آکویت', 'red salmon': 'ماهی قزل آلا قرمز', 'vichyssoise': 'vichyssoise', 'fondant': 'فوندانت', 'otter': 'سمور', 'beef': 'گوشت گاو', 
#  'orangutan': 'اورانگوتان', 'vermouth': 'شراب شیرین افسنطین', 'chub': 'چوب', 'meadowlark': 'میدلارک', 'peregrine': 'گل ریزه', 'blenny': 'بلنی',
#  'fingerling': 'انگشت زدن', 'bran': 'سبوس', 'ratatouille': 'راتاتویی', 'griffon': 'گریفون', 'beetle': 'سوسک', 'giraffe': 'زرافه',
#  'whippet': 'تازیانه زدن', 'chives': 'پیازچه', 'great ape': 'میمون بزرگ', 'yew': 'سرخدار', 'brew': 'دم کردن', 'tautog': 'توتوگ',
#  'sturgeon': 'ماهیان خاویاری', 'wrasse': 'wrasse', 'macaroni': 'ماکارونی', 'red drum': 'طبل قرمز', 'dumpling': 'پیراشکی', 'archaeopteryx': 'باستان شناسی', 
#  'agua': 'اب', 'bigeye': 'چشم درشت', 'scoter': 'کلاهبردار', 'codling': 'رمزگذاری', 'marlin': 'مارلین', 'spaniel': 'اسپانیل', 'muskrat': 'مشک',
#  'roebuck': 'سرخ کردن', 'macaque': 'ماکاک', 'insect': 'حشره', 'cedar': 'سرو', 'omnivore': 'همه چیزخوار', 'pandanus': 'پاندانوس', 
#  'glycerol': 'گلیسرول', 'ganoid': 'گانوئید', 'rhesus': 'راسوس', 'cannelloni': 'کانلونی', 'linnet': 'ملافه', 'rhea': 'رئا', 'hominid': 'انسان دوست', 
#  'patas': 'پاها', 'nectar': 'شهد', 'foxhound': 'روباه', 'firewater': 'آب گرم', 'candy': 'آب نبات', 'nighthawk': 'شاهین شبانه', 'bamboo': 'بامبو',
#  'wildfowl': 'مرغ وحشی', 'propanol': 'پروپانول', 'gazpacho': 'سوپ گازپاچو', 'trilobite': 'سه گانه', 'raita': 'مسیر', 'muntjac': 'muntjac', 
#  'kummel': 'جوجه', 'chicken': 'مرغ', 'daphnia': 'دافنی', 'ergosterol': 'ارگوسترول', 'dogwood': 'چوب سگ', 'chick': 'جوجه', 'duckling': 'جوجه اردک', 
#  'frog': 'قورباغه', 'moorhen': 'لعنتی', 'hen': 'بسیار', 'budgerigar': 'جوانه خوار', 'methanol': 'متانول', 'tody': 'سپس', 'salami': 'سالامی', 
#  'mayfly': 'ممکن است پرواز کند', 'grenadier': 'نارنجک', 'liquor': 'مشروب', 'cyclohexanol': 'سیکلوهگزانول', 'wireworm': 'کرم سیم', 'bird': 'پرنده', 
#  'iguanodon': 'ایگوانودون', 'dessert': 'دسر', 'hazel': 'فندق', 'sushi': 'سوشی', 'hardtack': 'سخت کوک', 'margate': 'حاشیه', 'domestic fowl': 'مرغ خانگی',
#  'scrod': 'اسکرول', 'mastiff': 'ماستیف', 'capuchin': 'کاپوچین', 'ouzo': 'اوزو', 'loach': 'لوچ', 'sake': 'منظور', 'pochard': 'پوچارد', 'grosbeak': 'منقل',
#  'prosimian': 'موافق', 'taffy': 'تافی', 'flummery': 'لودگی', 'mescal': 'مسکلی', 'bowfin': 'بوفین', 'poultry': 'طیور', 'mandrill': 'ماندریل', 
#  'meat': 'گوشت', 'spinach': 'اسفناج', 'moray': 'موری', 'bleu': 'آبی', 'belladonna': 'بلادونا', 'coney': 'رسوب کردن', 'capon': 'کاپون', 
#  'skink': 'چشم پوشی کردن', 'potage': 'سوپ', 'wassail': 'نوش', 'logwood': 'چوب جنگلی', 'pecan': 'اسپند', 'yellowfin': 'زرد', 'paca': 'پاکا', 
#  'fissiped mammal': 'پستاندار شکاف خورده', 'guillemot': 'گیلموت', 'capelin': 'کپلین', 'maple': 'افرا', 'retsina': 'رتسینا', 'basset': 'باس',
#  'tamarin': 'تامارین', 'duck': 'اردک', 'pumpkinseed': 'بذر کدو تنبل', 'phosphate': 'فسفات', 'anil': 'آنیل', 'plasmodium': 'پلاسمودیوم', 'rum': 'اتاق', 
#  'largemouth': 'دهان بزرگ', 'whiskey': 'نوشیدنی الکلی', 'baboon': 'بابون', 'squab': 'دست و پا زدن', 'borzoi': 'بورزوی', 'actinia': 'عمل جراحی',
#  'carbonado': 'کربنادو', 'spaghetti': 'ماکارونی', 'beagle': 'بیگل', 'fawn': 'حنایی', 'sisal': 'سیسال', 'firefly': 'کرم شب تاب', 'entellus': 'entellus', 
#  'chamois': 'چماق دار', 'skua': 'اسکوا', 'rosewood': 'گل محمدی', 'sparling': 'درخشان', 'slop': 'شیب', 'eucalyptus': 'اوکالیپتوس', 'barley': 'جو',
#  'guan': 'خاموش کردن', 'veery': 'منی', 'bologna': 'بولونیا', 'langur': 'طولانی', 'myna': 'مینا', 'killdeer': 'آدم کش', 'malmsey': 'مالمسی', 
#  'puffer': 'بافر', 'ladybug': 'کفشدوزک', 'maigre': 'لاغر', 'mare': 'بزرگ', 'bratwurst': 'براتورست', 'barbecue': 'باربیکیو', 'corgi': 'کورگی',
#  'couscous': 'دانه های کوسکوس', 'ratafia': 'راتافیا', 'grappa': 'گراپا', 'dalmatian': 'دالمیایی', 'jackdaw': 'کله پاچه', 'meal': 'وعده غذایی', 'adobo': 'تزئین',
#  'succotash': 'ساکوتاش', 'polecat': 'قطب شمال', 'mash': 'نه', 'coelacanth': 'همیار', 'lunch': 'ناهار', 'fish': 'ماهی', 'piranha': 'پیرانا', 
#  'imago': 'تصویر', 'cero': 'صفر', 'australopithecine': 'استرالوپیتسین', 'deer': 'گوزن', 'kinglet': 'سلطنت', 'tarpon': 'تارپون', 'solitaire': 'یک نفره', 
#  'solan': 'سولان', 'broiler': 'مرغ گوشتی', 'walking stick': 'چوب دستی', 'argentine': 'آرژانتین', 'gimlet': 'لقمه', 'pipit': 'گنجشک',
#  'hominoid': 'هومینوئید', 'goby': 'همسر', 'waterfowl': 'پرندگان آبزی', 'andrena': 'آندرنا', 'serin': 'سرد', 'nipa': 'توسط', 'balsa': 'قایق', 
#  'damselfish': 'خودخواه', 'milt': 'ارزن', 'kvass': 'کواس', 'wolfhound': 'سگ گرگ', 'dingo': 'دینگو', 'kirsch': 'کیرش', 'verdin': 'دادید',
#  'zinfandel': 'zinfandel', 'negus': 'منفی', 'nuthatch': 'مهره', 'cheese': 'پنیر', 'parakeet': 'پارکت', 'galago': 'کهکشان', 'old squaw': 'لقمه پیر',
#  'blackwood': 'چوب سیاه', 'eagle': 'عقاب', 'impala': 'ایمپالا', 'schnapps': 'اسکناس', 'cur': 'منحنی', 'gar': 'اصلاً', 'linguine': 'لینگوئینی',
#  'wafer': 'ویفر', 'lapin': 'لاپین', 'ape': 'بوزینه', 'falafel': 'فلافل', 'substance': 'ماده', 'screwdriver': 'پیچ گوشتی', 'blackbird': 'مرغ سیاه', 
#  'rickey': 'ریکی', 'caribou': 'کاریبو', 'pug': 'پاگ', 'wapiti': 'wapiti', 'mammal': 'پستاندار', 'raccoon': 'راکون', 'marc': 'مارک', 
#  'amberjack': 'کهربا', 'mamo': 'مامان', 'stickleback': 'عقب گرد', 'rauwolfia': 'راوولفیا', 'halibut': 'نوعی ماهی پهن بزرگ', 'whey': 'کشک',
#  'auklet': 'اوکلت', 'silverback': 'نقره ای', 'mocha': 'موکا', 'mouflon': 'موفلون', 'sheepshead': 'سر گوسفند', 'titmouse': 'تیتراژ', 'mousse': 'کف',
#  'flip': 'تلنگر', 'pilaf': 'پیلاو', 'dibranchiate': 'عوض کردن', 'weakfish': 'ماهی ضعیف', 'squid': 'ماهی مرکب', 'parr': 'پارل', 'pasta': 'ماکارونی',
#  'fettuccine': 'فتوسین', 'turnstone': 'سنگ تراش', 'colobus': 'کولوبوس', 'smallmouth': 'دهان کوچک', 'towhee': 'توله', 'bobolink': 'بوبولینک', 
#  'seahorse': 'اسب دریایی', 'pomfret': 'زیر پوستی', 'schnauzer': 'شناوزر', 'larch': 'لارچ', 'flax': 'کتان', 'swizzle': 'سوئیچ', 'sausage': 'سوسیس',
#  'pheasant': 'قرقاول', 'pastis': 'پاستیس', 'chestnut': 'فندق', 'koumiss': 'koumiss', 'homo': 'همو', 'broth': 'ابگوشت', 'loligo': 'لولیگو', 
#  'mustang': 'موستانگ', 'skipjack': 'اسکیپ جک', 'curlew': 'پیچش', 'home brew': 'دم خانگی', 'banger': 'کوبیدن', 'hickory': 'خارپشت', 
#  'kapok': 'می گیرم', 'simian': 'شبیه ساز', 'brant': 'اعطا کردن', 'grasshopper': 'ملخ', 'macon': 'ماکون', 'pyrethrum': 'پیرتروم', 'hind': 'hind', 
#  'monarch': 'monarch'}


In [ ]:
#@title dic for test
s=set()
with jsonlines.open('hypernym_conjunction_train.jsonl') as f:
  for line in f.iter():
    for c in line['question']['choices']:
      s.add(c["text"])
    line = line["question"]["stem"]
    line = line.replace(" are both a type of [MASK] ","")
    if line.find("[MASK]")!=-1:
      print(line)
      raise 1
    words = line[2:].split(" and a ")
    for word in words:
      s.add(word)
train_dict={w:translator.translate(w,dest='fa').text for w in s}
# translated = translator.translate('apple',dest='fa')
print(train_dict)

{'exarch': 'exarch', 'sulfamethazine': 'sulfamethazine', 'docent': 'docent', 'pisiform': 'pisiform', 'elevated railway': 'elevated railway', 'paganism': 'paganism', 'timothy': 'timothy', 'shard': 'shard', 'origanum': 'origanum', 'butt': 'butt', 'acrylic': 'acrylic', 'flying bridge': 'flying bridge', 'typhoid': 'typhoid', 'lansoprazole': 'lansoprazole', 'noise': 'noise', 'papule': 'papule', 'fennel': 'fennel', 'chi': 'chi', 'weeds': 'weeds', 'roadhouse': 'roadhouse', 'laundry': 'laundry', 'enalapril': 'enalapril', 'malleus': 'malleus', 'bolo': 'bolo', 'horse': 'horse', 'epistle': 'epistle', 'kingwood': 'kingwood', 'soldier': 'soldier', 'anticoagulant': 'anticoagulant', 'apomorphine': 'apomorphine', 'trophoblast': 'trophoblast', 'melancholy': 'melancholy', 'cactus': 'cactus', 'numeral': 'numeral', 'hemoglobin': 'hemoglobin', 'paperback book': 'paperback book', 'boldface': 'boldface', 'prothrombin': 'prothrombin', 'chloroform': 'chloroform', 'fibrin': 'fibrin', 'cassava': 'cassava', 'foru

In [ ]:
#@title save file

with jsonlines.open('size_comp_train.jsonl', mode='w') as writer:
    writer.write_all(train)
with jsonlines.open('size_comp_dev.jsonl', mode='w') as writer:
    writer.write_all(dev)

#multi-hop composition

In [ ]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/compositional_comparison/compositional_comparison_train.jsonl.gz
!gunzip compositional_comparison_train.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/compositional_comparison/compositional_comparison_dev.jsonl.gz
!gunzip compositional_comparison_dev.jsonl.gz

In [ ]:
#@title translate

def translate_object_comparison(data):
  res={}
  int_check=[]
  per_key={'A':0, 'B':1, 'C':2, 'first':'اول', 'second':'دوم', 'third':'سوم'}
  res['id']=data['id']
  res['label']=per_key[data['answerKey']]
  choices = []
  if len(data['question']['choices'])!=3 or (data['answerKey']!='A' and data['answerKey']!='B' and data['answerKey']!='C') :
    print(data)
    raise 1
  for choice in data['question']['choices']:
    if choice['label']!='A' and choice['label']!='B' and choice['label']!='C':
      print(data)
      raise 1
    choices.append({'label':per_key[choice['label']], 'text':per_key[choice['text']]})
  ages = data['question']['stem'].split()
  ages = ages[3],ages[7],ages[12]
  int_age = [int(i) for i in ages]
  stem = 'در مقایسه‌ی یک فرد '+ages[0]+' ساله با یک فرد '+ages[1]+' ساله و یک فرد '+ages[2]+' ساله، فرد [MASK] پیرترین است.'#بیشترین سن را دارد
  res['question']={'choices':choices, 'stem':stem}
  return res

with jsonlines.open('compositional_comparison_train.jsonl') as f:
  train=[]
  for line in f.iter():
    train.append(translate_object_comparison(line))

with jsonlines.open('compositional_comparison_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    dev.append(translate_object_comparison(line))


In [ ]:
#@title save file

with jsonlines.open('compositional_comp_train.jsonl', mode='w') as writer:
    writer.write_all(train)
with jsonlines.open('compositional_comp_dev.jsonl', mode='w') as writer:
    writer.write_all(dev)

#always-never ```translation_problem```


In [ ]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/coffee_cats_quantifiers/coffee_cats_quantifiers_train.jsonl.gz
!gunzip coffee_cats_quantifiers_train.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/coffee_cats_quantifiers/coffee_cats_quantifiers_dev.jsonl.gz
!gunzip coffee_cats_quantifiers_dev.jsonl.gz

--2021-04-17 14:05:54--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/coffee_cats_quantifiers/coffee_cats_quantifiers_train.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.101.170
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.101.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34929 (34K) [binary/octet-stream]
Saving to: ‘coffee_cats_quantifiers_train.jsonl.gz’

coffee_cats_quantif 100%[===================>]  34.11K  --.-KB/s    in 0.02s   

2021-04-17 14:05:54 (1.45 MB/s) - ‘coffee_cats_quantifiers_train.jsonl.gz’ saved [34929/34929]



In [ ]:
#@title translate
def translate_always_never(data):
  res={}
  per_key={'A':'الف', 'B':'ب', 'C':'ج', 'D':'د', 'E':'ه', 'often':'معمولا', 'never':'هیچگاه', 'rarely':'به ندرت', 'sometimes':'بعضی اوقات', 'always':'همواره'}
  res['id']=data['id']
  res['answerKey']=per_key[data['answerKey']]
  choices = []
  for choice in data['question']['choices']:
    choices.append({'label':per_key[choice['label']], 'text':per_key[choice['text']]})
  stem = translator.translate(data['question']['stem'],dest='fa').text
  res['question']={'choices':choices, 'stem':stem}
  return res

with jsonlines.open('coffee_cats_quantifiers_train.jsonl') as f:
  train=[]
  for line in f.iter():
    train.append(translate_always_never(line))
    
with jsonlines.open('coffee_cats_quantifiers_dev.jsonl') as f:
  dev=[]
  for line in f.iter():
    dev.append(translate_always_never(line))


In [ ]:
#@title save file
with jsonlines.open('always_never_train.jsonl', mode='w') as writer:
  writer.write_all(train)
with jsonlines.open('always_never_dev.jsonl', mode='w') as writer:
  writer.write_all(dev)

# breakable

In [ ]:
! pip install git+https://github.com/huggingface/transformers.git
! pip install git+https://github.com/huggingface/datasets.git
from IPython.display import clear_output 
!pip install -q sentencepiece

clear_output()
from transformers import BertTokenizer, BertForMaskedLM, AutoModelForMaskedLM, AutoTokenizer


In [ ]:
# from torch.nn import functional
# import torch
# امبرت: ساعت و تلویزیون/////////آینه چراغ گیتار شیشه  ساعت دوش موبایل تلویزیون سفال شانه
# توپ سکه پتو خودکار کلاه
# eng_model_checkpoint = "bert-base-uncased"
# model_checkpoint = "HooshvareLab/bert-base-parsbert-uncased"
# model_checkpoint = "HooshvareLab/albert-fa-zwnj-base-v2"
model_checkpoint = 'bert-base-multilingual-cased'
# model_checkpoint = 'HooshvareLab/roberta-fa-zwnj-base'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

a=tokenizer.encode('کتاب')
for i in a:
  print(tokenizer.decode(i),i)#۱۴۰۰۲ ۹۶۵۲


[CLS] 101
کتاب 19015
[SEP] 102


In [ ]:
from itertools import permutations

nbr=["توپ","سکه", "پتو", "خودکار", "کلاه","کتاب"]
br=["آینه","چراغ","گیتار","شیشه","ساعت","سفال"]
dataset=[]
# perm = permutations(nbr,3)
# for item1,item2, item3 in iter(perm):
#   for answer in br:
#     for i in range(4):
#       opts=[item1,item2, item3]
#       opts.insert(i,answer)
#       stem="یک "+opts[0]+"، یک "+opts[1]+"، یک "+opts[2]+" و یک "+opts[3]+" از پنجره پرتاب شدند. احتمال اینکه [MASK] بشکند از بقیه بیشتر است."
#       choices=[{"label":l,"text":t}for l,t in enumerate(opts)]
#       dataset.append({"question":{"stem":stem , "choices":choices},"label":i})


perm = permutations(br,3)
for item1,item2, item3 in iter(perm):
  for answer in nbr:
    for i in range(4):
      opts=[item1,item2, item3]
      opts.insert(i,answer)
      stem = "یک "+opts[0]+"، یک "+opts[1]+"، یک "+opts[2]+" و یک "+opts[3]+" از پنجره پرتاب شدند. احتمال اینکه [MASK] بشکند از بقیه کمتر است."
      choices=[{"label":l,"text":t}for l,t in enumerate(opts)]
      dataset.append({"question":{"stem":stem , "choices":choices},"label":i})


In [ ]:
from itertools import permutations

nbr=["محمد", "حجت", "طاهر",  "مهسا", "فاطمه", "آیدا" ]
br=["ناصر", "هاشم", "زهرا", "امیر" ,"کاوه" ,"حسین"]

dataset=[]
perm = permutations(nbr,3)
for item1,item2, item3 in iter(perm):
  for answer in br:
    for i in range(4):
      opts=[item1,item2, item3]
      opts.insert(i,answer)
      stem =opts[0]+ ' '+opts[1]+' ' +opts[2]+' ' +opts[3]+" [MASK] "
      choices=[{"label":l,"text":t}for l,t in enumerate(opts)]
      dataset.append({"question":{"stem":stem , "choices":choices},"label":i})


perm = permutations(br,3)
for item1,item2, item3 in iter(perm):
  for answer in nbr:
    for i in range(4):
      opts=[item1,item2, item3]
      opts.insert(i,answer)
      stem =opts[0]+ ' '+opts[1]+' ' +opts[2]+' ' +opts[3]+" [MASK] "
      choices=[{"label":l,"text":t}for l,t in enumerate(opts)]
      dataset.append({"question":{"stem":stem , "choices":choices},"label":i})


In [ ]:
len(dataset)

5760

In [ ]:
from IPython.display import clear_output
# ! pip install git+https://github.com/huggingface/datasets.git
!pip install jsonlines
clear_output()

In [ ]:
# with jsonlines.open("tmp.jsonl", mode="w") as writer:
#   writer.write_all(dataset)


In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(dataset)

msk = np.random.rand(len(df)) < 0.88
train = df[msk]
test = df[~msk]

len(train)
# df = df.groupby('fixed_label').sample(n = min(int(n/2),min(df.groupby('fixed_label').count()['label'])))


5066

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


df = pd.DataFrame(dataset)
print(df.shape)
X_train, X_test, y_train, y_test = train_test_split(df['question'], df['label'], test_size=0.1, random_state=0, stratify=df['label'])
train = pd.DataFrame({ 'question': X_train, 'label': y_train })
test = pd.DataFrame({ 'question': X_test, 'label': y_test })
# msk = np.random.rand(len(df)) < 0.97
# msk
test.groupby('label').count()


(2880, 2)


,question
label,
0,72
1,72
2,72
3,72


In [ ]:
!mkdir /content/data_chunks

In [ ]:

# write_path = 'breakable_dev.jsonl'#read_path.split('.')[0]+'_'+str(n)+'.jsonl'
with open('/content/data_chunks/breakable_dev2.jsonl', 'w') as file:
    test.to_json(file,orient = 'records', lines = True, force_ascii=False)
with open('/content/data_chunks/breakable_train2.jsonl', 'w') as file:
    train.to_json(file,orient = 'records', lines = True, force_ascii=False)
# print(df.to_json(orient='records', lines=True))

In [ ]:
# with open('breakable_nolang_train.jsonl','w') as f:
#   for i in datasets['train']:
#     f.write(str(i).replace("'","\"")+'\n')

# with open('breakable_nolang_dev.jsonl','w') as f:
#   for i in datasets['test']:
#     f.write(str(i).replace("'","\"")+'\n')
#     # pickle.dump(i,f)

In [ ]:
# from datasets import load_dataset

# datasets = load_dataset("json", data_files="tmp.jsonl").shuffle()
# # dataset_test = load_dataset('json', data_files='/content/ant-syn_dev.jsonl').shuffle()

# datasets = datasets['train'].train_test_split(760)
# datasets
# # datasets['validation'] = datasets['test']
# # datasets['test'] = dataset_test['train']

Using custom data configuration default-c497884ae1516696


0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-c497884ae1516696/0.0.0/45636811569ec4a6630521c18235dfbbab83b7ab572e3393c5ba68ccabe98264. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['question', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['question', 'label'],
        num_rows: 760
    })
})


# FARSNET


In [ ]:
!pip install zeep

     |████████████████████████████████| 100 kB 3.3 MB/s 
     |████████████████████████████████| 54 kB 1.7 MB/s 
     |████████████████████████████████| 45 kB 2.4 MB/s 


In [ ]:
from zeep import *
from requests import *
from zeep.wsse.username import UsernameToken


url = 'http://nlp.sbu.ac.ir:8180/WebAPI/services/SynsetService?WSDL'
userKey = 'd3f74906-3b91-11eb-8a1e-080027d731c1'

client = Client(url,wsse=UsernameToken('m.mirani',userKey))

print(client.service.Method1('Zeep', 'is cool'))


ConnectionError: ignored

In [ ]:
from zeep import Client

client = Client('http://www.webservicex.net/ConvertSpeed.asmx?WSDL')
result = client.service.ConvertSpeed(
    100, 'kilometersPerhour', 'milesPerhour')

assert result == 62.137

In [ ]:
from zeep import *
settings = Settings(strict=False, xml_huge_tree=True)
print(settings)

# SAMPLE from Dataset


In [ ]:
def process(line):
  for c in line['question']['choices']:
    if c['label'] == 0:
      if c['text'] == 'بزرگ':
        f = line['label']
      else:
        f = 1 - line['label']
  line['fixed_label'] = f
  return line
path = ['mbert_num_comp_train.jsonl','mbert_num_comp_train_nolang.jsonl','mbert_size_comp_train.jsonl','mbert_size_comp_train_nolang.jsonl']
# path = ['age_comp_train_nolang.jsonl','num_comp_train_nolang.jsonl','num_comp_train.jsonl','size_comp_train_nolang.jsonl']
sizes = [64,128,256,512,1024,2048,4096]
for n in sizes:
  for read_path in path:
    with jsonlines.open(read_path) as f:
      train=[]
      for line in f.iter():
        train.append(process(line))

    # print(min(int(n/2),int(df.shape[0]/2)))
    df = pd.DataFrame(train)
    df = df.groupby('fixed_label').sample(n = min(int(n/2),min(df.groupby('fixed_label').count()['label'])))

    write_path = read_path.split('.')[0]+'_'+str(n)+'.jsonl'
    with open('/content/data_chunks/'+write_path, 'w') as file:
        df.to_json(file,orient = 'records', lines = True, force_ascii=False)
    # print(df.to_json(orient='records', lines=True))


In [ ]:
!mkdir /content/data_chunks

In [ ]:
!mv /content/mbert_* /content/data_chunks/


In [ ]:
def process(line):
  for c in line['question']['choices']:
    if c['label'] == 0:
      if c['text'] == 'بود':
        f = line['label']
      else:
        f = 1 - line['label']
  line['fixed_label'] = f
  return line
path = ['ant-syn_train.jsonl']
sizes = [64,128,256,512,1024,2048,4096]
for n in sizes:
  for read_path in path:
    with jsonlines.open(read_path) as f:
      train=[]
      for line in f.iter():
        train.append(process(line))

    # print(min(int(n/2),int(df.shape[0]/2)))
    df = pd.DataFrame(train)
    df = df.groupby('fixed_label').sample(n = min(int(n/2),min(df.groupby('fixed_label').count()['label'])))

    write_path = read_path.split('.')[0]+'_'+str(n)+'.jsonl'
    with open('/content/data_chunks/'+write_path, 'w') as file:
        df.to_json(file,orient = 'records', lines = True, force_ascii=False)


In [ ]:
!mv age_comp_* /content/data_chunks/

In [ ]:
import jsonlines
path = ['/content/data_chunks/breakable_train.jsonl']
sizes = [64,128,256,512,1024,2048,4096]
for n in sizes:
  for read_path in path:
    
    with jsonlines.open(read_path) as f:
      train=[]
      for line in f.iter():
        train.append(line)

    # with open(read_path,encoding = 'utf-8-sig') as f:
    #   train = []
    #   for line in f.readlines():
    #     train.append(pickle.load(line))
    
    # print(min(int(n/2),int(df.shape[0]/2)))
    # print(train)
    df = pd.DataFrame(train)#,columns=['question','label'])
    # print(df.iloc[0][0])
    print(df)
    df = df.groupby('label').sample(n = min(int(n/4),min(df.groupby('label').count()['question'])))
    # print(df)

    write_path = 'breakable_train_'+str(n)+'.jsonl'
    with open('/content/data_chunks/'+write_path, 'w') as file:
        df.to_json(file,orient = 'records', lines = True, force_ascii=False)

    # write_path = read_path.split('.')[0]+'_'+str(n)+'.jsonl'
    # with open(write_path, 'w') as file:
    #     df.to_json(file,orient = 'records', lines = True, force_ascii=False)


                                               question  label
0     {'stem': 'مه‌رخ یک کلاه، یک گیتار، یک پتو و یک...      1
1     {'stem': 'مه‌رخ یک خودکار، یک پتو، یک آینه و ی...      2
2     {'stem': 'مه‌رخ یک کتاب، یک خودکار، یک توپ و ی...      3
3     {'stem': 'مه‌رخ یک کلاه، یک پتو، یک ساعت و یک ...      2
4     {'stem': 'مه‌رخ یک چراغ، یک کتاب، یک خودکار و ...      0
...                                                 ...    ...
5179  {'stem': 'مه‌رخ یک ساعت، یک کلاه، یک سکه و یک ...      0
5180  {'stem': 'فائزه یک توپ، یک ساعت، یک گیتار و یک...      0
5181  {'stem': 'فائزه یک سفال، یک سکه، یک گیتار و یک...      1
5182  {'stem': 'فائزه یک گیتار، یک پتو، یک شیشه و یک...      1
5183  {'stem': 'فائزه یک چراغ، یک آینه، یک کلاه و یک...      2

[5184 rows x 2 columns]
                                               question  label
0     {'stem': 'مه‌رخ یک کلاه، یک گیتار، یک پتو و یک...      1
1     {'stem': 'مه‌رخ یک خودکار، یک پتو، یک آینه و ی...      2
2     {'stem': 'مه‌رخ یک کتاب،

In [ ]:
!cd /content/data_chunks/
!zip -r num_comp.zip /content/data_chunks/mbert_num*

  adding: content/data_chunks/mbert_num_comp_train_1024.jsonl (deflated 87%)
  adding: content/data_chunks/mbert_num_comp_train_128.jsonl (deflated 86%)
  adding: content/data_chunks/mbert_num_comp_train_2048.jsonl (deflated 87%)
  adding: content/data_chunks/mbert_num_comp_train_256.jsonl (deflated 86%)
  adding: content/data_chunks/mbert_num_comp_train_4096.jsonl (deflated 87%)
  adding: content/data_chunks/mbert_num_comp_train_512.jsonl (deflated 87%)
  adding: content/data_chunks/mbert_num_comp_train_64.jsonl (deflated 85%)
  adding: content/data_chunks/mbert_num_comp_train_nolang_1024.jsonl (deflated 85%)
  adding: content/data_chunks/mbert_num_comp_train_nolang_128.jsonl (deflated 84%)
  adding: content/data_chunks/mbert_num_comp_train_nolang_2048.jsonl (deflated 85%)
  adding: content/data_chunks/mbert_num_comp_train_nolang_256.jsonl (deflated 84%)
  adding: content/data_chunks/mbert_num_comp_train_nolang_4096.jsonl (deflated 85%)
  adding: content/data_chunks/mbert_num_comp_tra

In [ ]:
!mkdir /content/data_chunks